In [20]:
using Plots
using JuMP
using Ipopt

In [2]:
#geometric mean of relative volitilies, useful for shortcut column
function geoMean(kLightKeyDist, kLightKeyBot, kHeavyKeyDist, kHeavyKeyBot)
    return(sqrt(kLightKeyDist*kLightKeyBot)/(kHeavyKeyDist*kHeavyKeyBot))
end

geoMean (generic function with 1 method)

In [3]:
#Fenske equation used in step 3 of column sizing to get the 
function fenske(xLightKeyDist, kLightKeyDist, xHeavyKeyDist, kHeavyKeyDist, xLightKeyBot, kLightKeyBot,
        xHeavyKeyBot, kHeavyKeyBot)
    α_avg = geoMean(kLightKeyDist, kLightKeyBot, kHeavyKeyDist, kHeavyKeyBot)
    n = log10(xLightKeyDist*xHeavyKeyBot/xLightKeyBot/xHeavyKeyDist) / log10(α_avg)
    return(n) #return number of trays
end

fenske (generic function with 1 method)

In [32]:
#Underwood equation for finding theta
function underwood(kDist, kBot, kHeavyKeyDist, kHeavyKeyBot, xFeed, qual, xDist)
    l = length(kDist)
    α = zeros(l)
    for i = 1:l
       α[i] = geoMean(kDist[i], kBot[i], kHeavyKeyDist, kHeavyKeyBot) 
    end
    
    num = sum(α .* xFeed)
    model = Model(with_optimizer(Ipopt.Optimizer))
    @variable(model, Θ)
    @NLobjective(model, Min, abs(num/sum(α[i] - Θ for i in 1:length(α))) + qual -1)
    optimize!(model)
    
    rmin = sum((α .* xDist) ./ (α .- value(Θ))) - 1
    return(rmin)
end

underwood (generic function with 1 method)

In [4]:
#Kirkbride function for finding optimal feed stage, uses ipopt solver
function kirkbride(B,D,xHKFeed,xLKFeed,xLKBot,xHKDist, nTot)
    t = ((B/D) * (xHKFeed/xLKFeed) * (xLKBot/xHKDist)^2)^0.206
    model = Model(with_optimizer(Ipopt.Optimizer))
    @variable(model, m) # where m is number of stages above feed
    @NLconstraint(model, 0.2 <= m <= nTot-0.2)
    @NLobjective(model, Min, abs(m/(nTot-m) - t))
    optimize!(model)
    return(value(m), nTot-value(m)) #return number stages above, number of stages below
end

kirkbride (generic function with 1 method)

### Part 1 
We begin by defining the light and heavy keys. We elected to use the Kabadi-Danner eos because it is designed for water-hydrocarbon systems particularly with dilute solutions. The heavy key is given as 1,2 dichloroethane. The light key was found using HYSYS K values. The next greatest K value is for water this is the light key. We assumed a feed basis of 100 kmol/hr.


![title](kValsFeed5b.png)

In [5]:
### define molar flows in feed, bottoms, and distillate by assuming splits given in problem statement
### and letting all HNK go to distillate and all LNK to the bottoms
feedBasis = 100
xFeed = [0.1493,0.5083,0.0007,0.0096,0.3321]
feedMolFlow = xFeed * feedBasis
distMolFlow = zeros(5)
botMolFlow = zeros(5)
distMolFlow[1] = feedMolFlow[1]; distMolFlow[2] = feedMolFlow[2]*0.005; distMolFlow[4] = feedMolFlow[4]*0.9; 
distMolFlow[5] = feedMolFlow[5]
botMolFlow[2] = feedMolFlow[2]*0.995; botMolFlow[3] = feedMolFlow[3]; botMolFlow[4] = feedMolFlow[4]*0.1;
xDist = zeros(5)
xBot = zeros(5)
for i = 1:5
    xDist[i] = distMolFlow[i]/sum(distMolFlow)
    xBot[i] = botMolFlow[i]/sum(botMolFlow)
end
print(xDist[2], " x val of HK in distilate " , xBot[4], " x val of LK in bottoms")

0.005159552277135864 x val of HK in distilate 0.0018919294428563409 x val of LK in bottoms

After using the above ratios in the performance data above and a pressure of 1100kPa in the bottoms(given) and 1090kPa in the distilate using the suggestion that for preliminary design we estimate 0.1 psi drop per tray with an estimated 10-15 trays, with a total condenser and partial reboiler. The minimum reflux ratio was then calculate to be 0.401. We multiplied by 1.27 which is in between 1.1 and 1.4 and got 0.51 which we used as the external reflux ratio, as seen below.
![title](shortcut_col_design_param.png)

We were then able to look at the performance tab to find the number of trays to use, 13, and the optimal feed stage 8. We were then ready to go with our full column simulation. We used the same feed stream composition, temperature and pressure and 13 stages. A once through regular HYSYS reboiler. 1100kPa for the reboiler, with a 10kPa drop through the reboiler, 1081 kPa for the condenser, with a 10kPa drop through the condenser, in accordance with the heuristic that a phase change in a heat exchanger should be done with a 10kPa drop. We used the condenser and reboiler temperature estimates from the shortcut column in the full column which were 68 and 183 degrees Celcius respectively. 0.51 for the reflux ratio and 2061 kgmol/hr in the distilate. We found that we were getting 96.7% recovery of the $12-ClC_2$ and so adjusted the distilate flow rate up to 2160 kgmol/hr. This resulted in a recovery rate of 95.1% and a purity in the bottoms stream of $12-ClC_2$ of 98.59%. The majority of the balance was water. If our customers will be satisfied with this purity then we should require no other seperations. If our customers need less water in their product we would suggest either a pressure swing column or with the purity already high a dehydrated salt could be used to scavenge the water. TRiiSO produces such chemicals specifically targeted at polyurthane production, and presumably our product will be used to produce PVC so this might be a good fit.
https://www.tri-iso.com/polyurethane-moisture-scavengers.html

The converged values for the design monitor and performance summary are shown below. Please see our flowsheet, included, to view all the column internals. 
![title](full_col_design_monitor.png)
![title](full_col_performance_summary.png)

Now we can use the K values from the full column to continue with the Fenske equation 

In [6]:
(0.9859*2074)/(4234*0.5083)

0.9501029253550254

In [18]:
α = [2.3, 4.2, 1.2, 8.2, 1.0]
sum(α)
sum(α .- 1)

11.899999999999999

In [23]:
α = [2.3, 4.2, 1.2, 8.2, 1.0]
xi = [0.2, 0.3, 0.4, 0.05, 0.05]
q = 0.79
num = 0
for i = 1:5
    num += α[i]*xi[i] 
end
model = Model(with_optimizer(Ipopt.Optimizer))
@variable(model, Θ)
@NLobjective(model, Min, abs(num/sum(α[i] - Θ for i in 1:length(α))) + q -1)
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

In [28]:
a = [1.0,2.0,3.0,4.0]
b = [1.0,2.0,3.0,4.0]
sum((a .* b) ./ (a .+ 1.0))

7.283333333333333

In [31]:
c = [1.0/2.0, 4.0/3.0, 9.0/4.0, 16.0/5.0]
sum(a .* b)

30.0